In [2]:
!pip install -U langchain langchain-community langchain-huggingface chromadb sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.4 MB/s eta 0:00:

**Running the embeddings.py file**

In [3]:
import pandas as pd
from langchain_community.vectorstores import Chroma # a vectore store to store embeddings and metadata
from langchain.docstore.document import Document # a standard format for storing text and metadata
from sentence_transformers import SentenceTransformer # used to turn text into embeddings using pre-trained modells like "all-mpnet-base-v2"
from langchain_huggingface import HuggingFaceEmbeddings
import os

# Load the dataset
data = pd.read_csv(r"/content/qna_dataset.csv")

# Creates a new column "Combined" Combining Question and Answer into a single text
data["Combined"] = data.apply(
    lambda row: f"question: {row['question']} answer: {row['answer']}", axis=1
)

# Create documents for Chroma, instead of plain text from the combined text, for more flexibility for more use cases and for working smoothly in langchain ecosystem
documents = [
    Document(page_content=row["Combined"], metadata={"question": row["question"], "answer": row["answer"]})
    for _, row in data.iterrows()
]

# Load the embedding model using LangChain's wrapper
embedding_function = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

try:
# Create the vector database using from_documents
    vector_db = Chroma.from_documents(
        documents,
        embedding=embedding_function,  # Correct: pass the wrapper, not just a function
        persist_directory="db"
    )
    print('Vector store created successfully')
except Exception as e:
    print(f"error: {e}")

print("Saving DB to:", os.path.abspath("db"))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector store created successfully
Saving DB to: /content/db


**Test**

In [5]:
from langchain.prompts import PromptTemplate
import pandas as pd

# Import your prompt template function
def get_prompt_template():
    template = """
You are a helpful e-commerce assistant. Use the following question-answer pairs (context) to help answer the user's question.

{context}

User's Question: {query}

Provide a clear and helpful answer based on the information above.
"""
    return PromptTemplate(input_variables=["context", "query"], template=template)

# Simulate user input
user_query = "How long does delivery take?"

# Use the vector database to retrieve relevant docs
retrieved_docs = vector_db.similarity_search(user_query)

# Build the context from retrieved documents
context = "\n".join([doc.page_content for doc in retrieved_docs])

# Get the prompt template and format it
prompt_template = get_prompt_template()
formatted_prompt = prompt_template.format(context=context, query=user_query)

# Print the formatted prompt
print(formatted_prompt)


You are a helpful e-commerce assistant. Use the following question-answer pairs (context) to help answer the user's question.

question: What is the discount on Condoms - Extra Time? answer: The discount on Condoms - Extra Time is 0.0%
question: What is the discount on For Boys - With Surprise? answer: The discount on For Boys - With Surprise is 0.0%
question: What is the discount on Rusk - Baby? answer: The discount on Rusk - Baby is 0.0%
question: What is the discount on Papad - Potato? answer: The discount on Papad - Potato is 0.0%

User's Question: How long does delivery take?

Provide a clear and helpful answer based on the information above.

